## Import funcitons and create a list of sessions that include the region of interest
to be able to run the notebook make sure of the following: you have installed the IBL enviroment alongside other dependencies as described in the README file of the repository.

In [3]:
import sys 
import os
from pathlib import Path
# Add the directory containing get_data.py to the Python path
sys.path.append(str(Path(os.getcwd()).resolve().parent)) # add the root of the project to the python path
from extraction_data import get_pid_eid_pairs
pid_eid_pairs = get_pid_eid_pairs(only_passive=True, regions = ['VISp'])

Connected to https://openalyx.internationalbrainlab.org as user "intbrainlab"
Total number of possible probe insertions: 73
Number of sessions without passive data: 23
Number of sessions with passive data: 50


# Input Parameters

## Preprocessing Parameters
-   **`min_contrast`**: Minimum contrast value used to filter the trials.
-   **`t_bin`**: Size of the time bin (in seconds) used for calculating firing rates.
-   **`pre_stim`** and **`post_stim`**: Time (in seconds) before and after stimulus onset to consider when calculating firing rates. ATTENTION: pre_stim window is used for z-score baseline correction of the firing rates.
-   **`min_time`**: Minimum time (in seconds) after stimulus onset to include in the final results which goes in decoding procedures. 
-   **`filter_regions`**: List of brain regions to include in the analysis.
-   **`only_good_clusters`**: Boolean flag indicating whether to filter clusters to include only those classified as "good".
-   **`contrast_filter`** and **`probabilityLeft_filter`**: Lists specifying valid contrast and probability left values for trial filtering. the probabilityLeft_filter is applied only for active segment.

## Decoding Parameters
-   **`test_strategy`**: Decoding procedure to use ('passive', 'active', 'both').
    - **passive**: Train on 80% of passive and test decoding on the 20% of passive untrained data (using k-fold).
    - **active**: Train on passive data and test on active data.
    - **both**: Train on 80% of passive data and test on 20% of passive and all active data.
-   **`feature_selection`**: Feature selection method to use ('pca', 'average_clusters'). (average_clusters gives better results).
-   **`n_components`**: Number of components to retain for PCA.
-   **`n_permutations`**: Number of permutations used for null distribution generation.
-   **`n_folds`**: Number of folds for cross-validation.
-   **`n_time_bins`**: Number of time bins to include in the classification. If None, use all time bins.


In [4]:
from functions import Apply_decoding, monitor_job_status
import submitit
import pandas as pd 
import numpy as np


PARAMS_preprocess = {'min_contrast': 1,  
          't_bin': 0.02, 'pre_stim': 0.5, 'post_stim': 1.0, 'min_time': 0,
            'filter_regions':  ['VISp'], 'only_good_clusters': False,
              'probabilityLeft_filter': [0.5], 
                'contrast_stim_filter': [0, 1]}
PARAMS_decoding = { 'test_strategy': 'active', 'n_components': 5, 'feature_selection': 'average_clusters', 'n_time_bins': -1, 'n_permutations': 1000, 'n_folds': 5}

PARAMS_submitit = {'maxjob': 60  ,'memory': 15 , 'time': 300, 'partition': 'CPU', 'cpus_per_task': 1}

save_paths = [f"results/{pid}_V1_TPassive_TActive.pkl" for pid, _ in pid_eid_pairs]



#  decoding for right/left selectivity using submitit (CRLN server)

In [ ]:
import submitit
def submit_decode(pid_eid_pair, save_path):
    pid, eid = pid_eid_pair
    Apply_decoding(eid, pid, PARAMS_preprocess, PARAMS_decoding, save=True, save_path=save_path)

# Create an executor
executor = submitit.AutoExecutor(folder='submitit_debug')
executor.update_parameters(slurm_array_parallelism=PARAMS_submitit['maxjob'],mem_gb=PARAMS_submitit['memory'],timeout_min= PARAMS_submitit['time'],slurm_partition= PARAMS_submitit['partition'],cpus_per_task=PARAMS_submitit['cpus_per_task'])
# Submit all jobs using map_array
all_jobs = executor.map_array(submit_decode, pid_eid_pairs, save_paths)
monitor_job_status(all_jobs)


# serial decoding (for local computer)

In [ ]:
i=0
for pid, eid in pid_eid_pairs:
    i+=1
    print(f"Decoding {pid} {i}/{len(pid_eid_pairs)}")
    save_path = f"results/{pid}_v1a_TPassive_TActive.pkl"
    Apply_decoding(eid, pid, PARAMS_preprocess, PARAMS_decoding, save=True, save_path=save_path)
    

    